<a href="https://colab.research.google.com/github/francesco-source/DeepLearning/blob/main/mnistDense.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mnist classification with NNs
A first example of a simple Neural Network, applied to a well known dataset.

In [30]:
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Model
from tensorflow.keras import utils
import numpy as np

Let us load the mnist dataset

In [31]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [32]:
print(x_train.shape)
print("pixel range is [{},{}]".format(np.min(x_train),np.max(x_train)))

(60000, 28, 28)
pixel range is [0,255]


We normalize the input in the range [0,1]

In [33]:
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

x_train = np.reshape(x_train,(60000,28*28))
x_test = np.reshape(x_test,(10000,28*28))

The output of the network will be a proability distribution over the different categories. Similarly, we generate a ground truth distribution, and the training objective will consist in minimizing their distance (categorical crossentropy). The ground truth distribution is the so called "categorical" distribution: if x has label l, the corresponding categorical distribution has probaility 1 for the category l, and 0 for all the others.

In [34]:
#we use the categoricl rapresentation of the class
print(y_train[0])
y_train_cat = utils.to_categorical(y_train)
print(y_train_cat[0])
y_test_cat = utils.to_categorical(y_test)
# This is what we expect that the network will produce. 

5
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]


Our first Netwok just implements logistic regression

In [35]:
xin = Input(shape=(784)) # First layer, only one layer with this shape.
# 10 is the number of outputs of the dense layer, activation is the softmax(generalization of the logistic function).
res = Dense(10,activation='softmax') (xin) # We apply the dense layer to the input layer. We create the layer.
mynet = Model(inputs=xin,outputs=res) #inputs = could be a list of inputs and also outputs can be a list

In [36]:
mynet.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 784)]             0         
                                                                 
 dense_6 (Dense)             (None, 10)                7850      
                                                                 
Total params: 7,850
Trainable params: 7,850
Non-trainable params: 0
_________________________________________________________________


The None means that the beach dimension is unspecified. The network is able to work with any betch dimension. We have a dense network so we know that the parameters are all the possible connections. 784*10 + 1*10 biases links. 

Now we need to compile the network.
In order to do it, we need to pass two mandatory arguments:


*   the **optimizer**, in charge of governing the details of the backpropagation algorithm
*   the **loss function**

Several predefined optimizers exist, and you should just choose your favourite one. A common choice is Adam, implementing an adaptive lerning rate, with momentum

Optionally, we can specify additional metrics, mostly meant for monitoring the training process.

Th optimizer is usually a variant of SGD.

In [37]:
mynet.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

Finally, we fit the model over the trianing set. 

Fitting, just requires two arguments: training data e ground truth, that is x and y. Additionally we can specify epochs, batch_size, and many additional arguments.

In particular, passing validation data allow the training procedure to measure loss and metrics on the validation set at the end of each epoch.

In [38]:
# batch size, amount of data that we want to process in parallel. Largest batch size possible is usually a good idea. 
# Here we are using test set as validation. Is cheading but only for teaching purpouses we do so. 
mynet.fit(x_train,y_train_cat, shuffle=True, epochs=10, batch_size=32,validation_data=(x_test,y_test_cat))

Epoch 1/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.4701 - accuracy: 0.8771 - val_loss: 0.3096 - val_accuracy: 0.9150
Epoch 2/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3040 - accuracy: 0.9154 - val_loss: 0.2823 - val_accuracy: 0.9205
Epoch 3/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2831 - accuracy: 0.9213 - val_loss: 0.2737 - val_accuracy: 0.9254
Epoch 4/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2727 - accuracy: 0.9237 - val_loss: 0.2686 - val_accuracy: 0.9260
Epoch 5/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2664 - accuracy: 0.9261 - val_loss: 0.2690 - val_accuracy: 0.9247
Epoch 6/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2616 - accuracy: 0.9278 - val_loss: 0.2644 - val_accuracy: 0.9274
Epoch 7/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2584 - accuracy: 0.9284 - val_loss: 0.2660 - val_accuracy:

Now we try to add more layers and do a better logistic regression. We can so add intermidiet layers. Now we use tree layers. And i can choose the size. Softmax is usally used only for the final layes. Relu is used for intermediet layers. You can specify an activation as an additional layer if you want.

In [39]:
xin = Input(shape=(784))
x = Dense(128,activation='relu')(xin)
res = Dense(10,activation='softmax')(x)
# X2 = Dense(100)(x)
# X3 = Activation(activation = "relu")(X2)

mynet2 = Model(inputs=xin,outputs=res)

In [40]:
mynet2.summary()

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 784)]             0         
                                                                 
 dense_7 (Dense)             (None, 128)               100480    
                                                                 
 dense_8 (Dense)             (None, 10)                1290      
                                                                 
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


In [41]:
mynet2.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [42]:
mynet2.fit(x_train,y_train_cat, shuffle=True, epochs=10, batch_size=32,validation_data=(x_test,y_test_cat))

Epoch 1/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2576 - accuracy: 0.9265 - val_loss: 0.1317 - val_accuracy: 0.9614
Epoch 2/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1148 - accuracy: 0.9667 - val_loss: 0.1025 - val_accuracy: 0.9694
Epoch 3/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0780 - accuracy: 0.9768 - val_loss: 0.0895 - val_accuracy: 0.9719
Epoch 4/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0588 - accuracy: 0.9823 - val_loss: 0.0833 - val_accuracy: 0.9742
Epoch 5/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0442 - accuracy: 0.9868 - val_loss: 0.0833 - val_accuracy: 0.9751
Epoch 6/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0363 - accuracy: 0.9890 - val_loss: 0.0696 - val_accuracy: 0.9793
Epoch 7/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0285 - accuracy: 0.9913 - val_loss: 0.0727 - val_accuracy:

An amazing improvement. WOW!

In [43]:
mynet.save_weights("my_file")
mynet2.save_weights("my_file")

# Exercises

1.   Add additional Dense layers and check the performance of the network
2.   Replace 'relu' with different activation functions
3. Adapt the network to work with the so called sparse_categorical_crossentropy
4. the fit function return a history of training, with temporal sequences for all different metrics. Make a plot.

\\